In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np

pd.set_option('display.max_columns', None)


In [3]:
prepped_df = pd.read_csv('prepped_data.csv')

X = prepped_df.drop(columns=['accepted']).copy()
y = prepped_df['accepted'].copy()

prepped_df.head()

,discussions_count,comments_count,movie_count,screenshot_count,awards_count,file_size_mb,change_notes,certified_compatible,creator_count,featured_collection_count,visitors_count,favorites_count,accepted,class_count,class_scout,class_soldier,class_pyro,class_demoman,class_heavy,class_engineer,class_medic,class_sniper,class_spy,item_slot_headgear,item_slot_weapon,item_slot_taunt,item_slot_misc,submitted_by_submitter
0,0,154,0,6,0,4.971,7,1,2,115,5492,428,1,5.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,380.0
1,0,40,0,4,0,2.958,4,1,2,46,2218,173,1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,380.0
2,0,238,0,2,0,4.277,1,0,4,100,11905,859,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,153.0
3,0,237,0,8,0,4.567,3,0,2,91,10085,650,1,9.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,114.0
4,3,773,0,0,0,0.572,1,0,1,17,33200,102,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=13)
lg1 = LogisticRegression(max_iter=100000, random_state=13,
                         class_weight={0: 1, 1: 5})
lg1.fit(X_train, y_train)

y_pred = lg1.predict(X_test)

print(f'Confusion Matrix: \n{confusion_matrix(y_test, y_pred)}')

print(f'Accuracy Score: {accuracy_score(y_test,y_pred)}')
print(f'Recall score: {recall_score(y_test,y_pred)}')
print(f'F1 score: {f1_score(y_test,y_pred)}')
print(f'Area Under Curve: {roc_auc_score(y_test, y_pred)}')


Confusion Matrix: 
[[5672  425]
 [ 245  288]]
Accuracy Score: 0.8989441930618401
Recall score: 0.5403377110694184
F1 score: 0.4622792937399679
Area Under Curve: 0.7353156490397117
